To plot the APD90 difference in the domain of the 3 interested parameters

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go

import modelling

In [2]:
category = False

In [4]:
# 3D plot in parameter space
# Plot for known drugs
param_lib = modelling.BindingParameters()
drug_list = param_lib.drug_compounds

SA_model = modelling.SensitivityAnalysis()
param_names = SA_model.param_names

starting_param_df = pd.DataFrame([1] * 5, index=param_names).T
ComparisonController = modelling.ModelComparison(starting_param_df)

discrete_colors = ['red', 'blue', 'black']
APD_diff_label = ['similar', 'SD higher', 'CS higher']

fig = go.Figure()

color_code_list = []
Vhalf_list = []
Kmax_list = []
Ku_list = []
RMSError_list = []

for drug in drug_list:
    saved_data_dir = '../../../simulation_data/binding_kinetics_comparison/' + \
        drug + '/Milnes/'
    APD_trapping = pd.read_csv(saved_data_dir + 'CiPA_APD_pulses1000.csv')
    APD_conductance = pd.read_csv(
        saved_data_dir + 'conductance_APD_pulses1000.csv')

    drug_conc = APD_trapping['drug concentration'].values.tolist()
    APD_trapping = [max(APD_trapping.loc[i].values.tolist()[1 + 998:-1])
                    for i in range(APD_trapping.shape[0])]
    APD_conductance = [max(APD_conductance.loc[i].values.tolist()[1 + 998:-1])
                       for i in range(APD_conductance.shape[0])]
    RMSError = ComparisonController.compute_RMSE(APD_trapping,
                                                 APD_conductance)
    MAError = ComparisonController.compute_MAE(APD_trapping,
                                               APD_conductance)

    if RMSError < 100:
        color_code = 0
    elif MAError > 0:
        color_code = 1
    else:
        color_code = 2

    color_code_list.append(color_code)
    Vhalf_list.append(param_lib.binding_parameters[drug]['Vhalf'])
    Kmax_list.append(param_lib.binding_parameters[drug]['Kmax'])
    Ku_list.append(param_lib.binding_parameters[drug]['Ku'])
    RMSError_list.append(RMSError * MAError / np.abs(MAError))

if category:
    fig.add_trace(
        go.Scatter3d(
            x=Vhalf_list,
            y=Kmax_list,
            z=Ku_list,
            mode='markers',
            marker_symbol='diamond',
            name='drugs',
            marker=dict(
                color=color_code_list,
                colorscale=discrete_colors
            )
        )
    )
else:
    fig.add_trace(
        go.Scatter3d(
            x=Vhalf_list,
            y=Kmax_list,
            z=Ku_list,
            mode='markers',
            marker_symbol='diamond',
            name='drugs',
            marker=dict(
                color=RMSError_list,
                colorscale='RdBu'
            )
        )
    )

saved_data_dir = '../../../simulation_data/sensitivity_analysis/'
filename = 'SA_allparam_0_copy.csv'
df = pd.read_csv(saved_data_dir + filename,
                 header=[0, 1], index_col=[0],
                 skipinitialspace=True)

# Exploring the space
Vhalf_range = df['param_values']['Vhalf'].values
Kmax_range = df['param_values']['Kmax'].values
Ku_range = df['param_values']['Ku'].values

RMSError = df['RMSE']['RMSE'].values
MAError = df['MAE']['MAE'].values

param_id = df['param_id']['param_id'].values
color_code_list = []
for i in range(len(param_id)):
    if RMSError[i] < 100:
        color_code = 0
    elif MAError[i] > 0:
        color_code = 1  # SD higher
    else:
        color_code = 2  # CS higher
    color_code_list.append(color_code)

if category:
    fig.add_trace(
        go.Scatter3d(
            x=Vhalf_range,
            y=Kmax_range,
            z=Ku_range,
            mode='markers',
            name='space',
            marker=dict(
                color=color_code_list,
                colorscale=discrete_colors,
                opacity=0.5,
                size=5
            )
        )
    )
else:
    color = RMSError * MAError / np.abs(MAError)
    fig.add_trace(
        go.Scatter3d(
            x=Vhalf_range,
            y=Kmax_range,
            z=Ku_range,
            mode='markers',
            name='space',
            marker=dict(
                color=color,
                colorscale='RdBu',
                opacity=0.5,
                size=5
            )
        )
    )

fig.update_layout(scene = dict(
                    xaxis_title='Vhalf',
                    yaxis_title='Kmax',
                    zaxis_title='Ku',
                    xaxis = dict(
                         backgroundcolor="rgb(159, 214, 77)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                         backgroundcolor="rgb(159, 214, 77)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",
                         dtick=1,
                         type='log'),
                    zaxis = dict(
                         backgroundcolor="rgb(159, 214, 77)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",
                         dtick=1,
                         type='log')))

fig.show()